In [0]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/L6")

In [5]:
!pip install deepctr
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat,get_feature_names

     |████████████████████████████████| 92kB 4.6MB/s 


In [6]:
df = pd.read_csv('./movielens_data/ratings.dat', sep = '::',header = None)
movies = pd.read_csv('./movielens_data/movies.dat', sep = '::',header = None)
users = pd.read_csv('./movielens_data/users.dat', sep = '::',header = None)
df.columns = ['userId','movieId','Rating','timestamp']
movies.columns = ['movieId','name','tag']
users.columns = ['userId','sex','age','occupation','zipcode']
df_tmp = pd.merge(df, movies, on = 'movieId')
data_whole = pd.merge(df_tmp,users, on = 'userId')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel 

In [7]:
data_whole.head()

,userId,movieId,Rating,timestamp,name,tag,sex,age,occupation,zipcode
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [9]:
data_whole.shape

(1000209, 10)

In [0]:
sparse_features = ["movieId", "userId", "sex", "age", "occupation", "zipcode"]
target = ['Rating']

In [0]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data_whole[feature] = lbe.fit_transform(data_whole[feature])

In [0]:
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data_whole[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [0]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data_whole, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [29]:
# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=True, validation_split=0.2)

Train on 640133 samples, validate on 160034 samples
Epoch 1/10
640133/640133 [==============================] - 17s 26us/sample - loss: 1.0073 - mean_squared_error: 1.0066 - val_loss: 0.8353 - val_mean_squared_error: 0.8341
Epoch 2/10
640133/640133 [==============================] - 16s 26us/sample - loss: 0.8156 - mean_squared_error: 0.8138 - val_loss: 0.8063 - val_mean_squared_error: 0.8040
Epoch 3/10
640133/640133 [==============================] - 17s 26us/sample - loss: 0.7962 - mean_squared_error: 0.7932 - val_loss: 0.7958 - val_mean_squared_error: 0.7923
Epoch 4/10
640133/640133 [==============================] - 16s 25us/sample - loss: 0.7791 - mean_squared_error: 0.7750 - val_loss: 0.7945 - val_mean_squared_error: 0.7898
Epoch 5/10
640133/640133 [==============================] - 16s 25us/sample - loss: 0.7655 - mean_squared_error: 0.7602 - val_loss: 0.7827 - val_mean_squared_error: 0.7769
Epoch 6/10
640133/640133 [==============================] - 16s 25us/sample - loss: 0.75

In [0]:
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)

In [0]:
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)

In [0]:
rmse = mse ** 0.5

In [33]:
print("test RMSE", rmse)

test RMSE 0.8782938005018593


In [34]:
pred_ans

array([[3.5270753],
       [3.3718133],
       [3.1850855],
       ...,
       [3.068005 ],
       [3.8828633],
       [4.605917 ]], dtype=float32)

In [35]:
test[target].values

array([[1],
       [4],
       [4],
       ...,
       [3],
       [4],
       [5]])